# 数学计算
当今大模型面临的最困难任务之一是需要某种形式的推理的任务。推理是最具吸引力的领域之一，因为有了推理，就可以从LLM中涌现出各种复杂的应用类型。

目前，涉及数学能力的推理任务已经有了一些改进。 对于当前的大型语言模型来说，执行推理任务可能会有一些难度，但是在很多官方发布的Instruct模型，训练数据中都会有很多包含思考推理过程的数据，尤其是在数学推理模块，因为这通常意味着模型最终在各个领域的评测榜单里能占领高位。现在，我们将介绍几个基本示例来展示数学计算模块。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
prompt_1="9.11和9.8哪个更大？"

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt_1))

{'response': '比较日期9.11和9.8的大小，实际上是比较9月11日和9月8日。\n\n从数值上看，9月11日在9月8日之后，因此9.11比9.8更大。', 'generated_tokens_count': 53}


因为9.11也可以表示成日期，因此在对比的时候可能会出错，这不仅要求我们将问题明确，而且因为是数学问题，为了提高回答的准确性，可以在提示词中使用CoT

In [6]:
prompt_2="从数学角度分析9.11和9.8哪个更大？让我们一步一步思考"

In [7]:
print(inference(model_instruct,tokenizer_instruct,prompt_2))

{'response': '要比较9.11和9.8这两个数字，我们可以通过数学的方法来进行分析。\n\n1. **直接比较法**：最直观的方法就是直接观察这两个数：\n   - 9.11 和 9.8\n\n2. **将它们放在同一数位上进行比较**：如果我们将两个数的小数部分都转换为相同的小数位数，以便更清晰地比较整数部分和小数部分。\n   - 9.11 可以写成 9.110（在末尾添加了一个零）\n   - 9.8 可以写成 9.80（同样在末尾添加了一个零）\n\n3. **比较整数部分**：两个数的整数部分都是9，因此继续比较小数部分。\n   - 9.110 的小数部分是 0.11\n   - 9.80 的小数部分是 0.80\n\n4. **比较小数部分**：逐位比较 0.11 和 0.80：\n   - 第一位小数：1 和 8。显然，1 < 8。\n   \n因为 0.11 < 0.80，所以 9.11 < 9.8。\n\n因此，从数学的角度分析，9.8 比 9.11 更大。', 'generated_tokens_count': 297}


从结果上看，不仅思考的步骤要比单纯的提问要更多，并且从整数部分到小数，都进行了细致的排查，从而形成了正确的思考链路，即使多次实验，最终结果出错的概率也大幅减少，这就是思维链的魅力所在，仅用一行提示就能改善模型性能，因为事实上一句简单的提示激活了模型的思考反思能力，这或许就是我们常说的“三思而后行”。